In [444]:
import cv2
import numpy as np

def select_white_yellow(image):
    hls = convert_hls(image)
    # white color mask
    lower = np.uint8([  0, 200,0])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(hls, lower, upper)
    
    hsv = convert_hsv(image)
    # yellow color mask HSV
    lower = np.uint8([ 40, 20, 0])
    upper = np.uint8([ 110, 255, 255])
    #lower = np.uint8([ 30, 20, 0])
    #upper = np.uint8([ 80, 100, 100])
    
    yellow_mask = cv2.inRange(hsv, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)

    return cv2.bitwise_and(image, image, mask = mask)
def convert_hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
def convert_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

def select_region(image):
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.35, rows*0.95]
    top_left     = [cols*0.42, rows*0.6]
    bottom_right = [cols*0.6, rows*0.95]
    top_right    = [cols*0.50, rows*0.6] 
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    mask = np.zeros_like(image)
    if len(img.shape)==3:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2])      
    return cv2.bitwise_and(image, mask)

In [465]:

 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#vedioname = input("vedio name: ")
vedioname = 't1.mp4'
cap = cv2.VideoCapture(vedioname)
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
# Capture frame-by-frame
    ret, frame = cap.read()
    

    # main operation goes here
    if ret == True:
        frame = cv2.resize(frame, (640, 480))
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        r,c,n = frame.shape
        midx = int(c/2)
        midy = int(r/2)
        offsetx = 80
        offsety = 100
        frame = frame[midx-offsetx:midx+offsetx, midy-offsety:midy+offsety]
        
        
        #equlizer in rgb
        img_yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        # equalize the histogram of the Y channel
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        # convert the YUV image back to RGB format
        img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        
        #converting grayscale and resize
        img = select_white_yellow(img)
        
        #img = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        cropx,cropy = gray.shape

        
        #Equlization cliplimit 1 subtle, 3,4 aggressive
        #clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(2,2))
        #gray = clahe.apply(gray)
        #Equlization 
        #gray = cv2.equalizeHist(gray)
        
        #find features
        #corners = cv2.goodFeaturesToTrack(gray,25,0.01,10)
        #corners = np.int0(corners)
        #draw features
        #for i in corners:
        #    x,y = i.ravel()
        #    cv2.circle(gray,(x,y),3,255,-1)
        
        #Threshhold
        #retse, gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        
        #birds eye transformation
        ofho = 85
        src = np.float32([[0, cropx], [cropy, cropx], [0, 0], [cropy, 0]])
        dst = np.float32([[ofho, cropx], [cropy-ofho, cropx], [0, 0], [cropy, 0]])
        M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
        Mi = cv2.getPerspectiveTransform(dst, src) # The transformation matrix
        gray = cv2.warpPerspective(gray, M, (cropy, cropx))
        
        #cutof = 60
        #gray = gray[0:cropx,cutof:cropy-cutof]
        #gray = cv2.warpPerspective(gray, Mi, (cropy, cropx))
                
        #filter region
        gray = select_region(gray)
        
        #canny edge detection
        gray = cv2.Canny(gray,80,150,apertureSize = 3)
        
        #dilation process
        kernel = np.ones((2,2),np.uint8)
        gray = cv2.dilate(gray,kernel,iterations = 1)
        
        #threshold
        retse, gray = cv2.threshold(gray,100,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        
        #lsd line detection
        lsd = cv2.createLineSegmentDetector(0)
        dlines = lsd.detect(gray)[0]
        gray = lsd.drawSegments(gray,dlines)
        
        gray = cv2.resize(gray,(640,480))
        cv2.imshow('Frame',gray) 
        #show video
        #cv2.imshow('Frame',gray)    
        if cv2.waitKey(50000) & 0xFF == ord('q'):
            break
    else:
        break
    # Press Q on keyboard to exitq
    


# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()